## 基于prompt——tuning的两种形式，soft和hard

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer

In [2]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [5]:
def process_fun(example):
    MAX_LENGTH = 256
    input_ids,attention_mask,labels = {},{},{}
    instruction = "\n".join(["Human:" + example["instruction"],example["input"]]).strip() + "\n\nAssistant:"
    instruction = tokenizer(instruction)
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100]*len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_fun, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human:解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant:4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [8]:
tokenizer.decode(list(filter(lambda x:x!=-100,tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [9]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")

# Prompt tuning
## PEFT Step1 配置文件
### soft_prompt的配置方法

In [25]:
from peft import PromptTuningConfig,get_peft_model,TaskType,PromptTuningInit,PeftModel
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,num_virtual_tokens=10)
config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=10, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, prompt_tuning_init=<PromptTuningInit.RANDOM: 'RANDOM'>, prompt_tuning_init_text=None, tokenizer_name_or_path=None)

### hard_prompt


In [11]:
from peft import PromptTuningConfig,get_peft_model,TaskType,PromptTuningInit,PeftModel
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
                            prompt_tuning_init=PromptTuningInit.TEXT,
                            prompt_tuning_init_text="下面是一段人与机器人的对话。",
                            num_virtual_tokens=len(tokenizer("下面是一段人与机器人的对话。")["input_ids"]),
                            tokenizer_name_or_path="Langboat/bloom-1b4-zh")
config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=8, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, prompt_tuning_init=<PromptTuningInit.TEXT: 'TEXT'>, prompt_tuning_init_text='下面是一段人与机器人的对话。', tokenizer_name_or_path='Langboat/bloom-1b4-zh')

In [12]:
model = get_peft_model(model,config)

In [13]:
model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(46145, 2048)
      (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
            (dense): Linear(in_features=2048, out_features=2048, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
          )
        )
      )

In [14]:
model.print_trainable_parameters()

trainable params: 16,384 || all params: 1,303,128,064 || trainable%: 0.0012572824154909767


In [15]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=20,
    save_total_limit=3  # 只要最新的3个，之前的会被自动删除
)

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

/var/folders/vt/2rwrwr1j1hn9724wbbsr7jm80000gn/T/ipykernel_3174/1988846643.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

/Users/liuzhen/Documents/PEFT_Study/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.670600
20,2.707300
30,2.733400
40,2.690000
50,2.630500
60,2.633800
70,2.525200
80,2.590100


KeyboardInterrupt: 

# 加载训练好的模型

In [18]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
peft_model = PeftModel.from_pretrained(model=model, model_id="./chatbot/checkpoint-80/")

In [19]:
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"
peft_model = peft_model.to(device)

ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))

RuntimeError: The expanded size of the tensor (13) must match the existing size (21) at non-singleton dimension 2.  Target sizes: [16, 13, 13].  Tensor sizes: [16, 1, 21]

In [23]:
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"
import torch

import torch

# 1. 准备输入
text = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
ipt = tokenizer(text, return_tensors="pt").to(device)
input_ids = ipt['input_ids']
batch_size = input_ids.shape[0]

# 2. 提取 Prompt Embedding (保持不变)
prompt_embeds = peft_model.prompt_encoder.default.embedding.weight
prompt_embeds = prompt_embeds.unsqueeze(0).expand(batch_size, -1, -1)

# 3. 核心修复：正确获取 Bloom 的 Embedding
with peft_model.disable_adapter():
    # --- 修改点在这里 ---
    # 之前报错是因为写了 peft_model.base_model.model
    # 既然报错说 'BloomForCausalLM' 没有 .model，那说明 peft_model.base_model 本身就是我们要找的对象！
    base_model = peft_model.base_model 
    
    # 这是一个通用的方法，无论模型内部叫 transformer 还是 model 都能拿到
    embedding_layer = base_model.get_input_embeddings() 
    
    # 获取原始文本的 Embedding
    raw_embeds = embedding_layer(input_ids)
    # --- 修改结束 ---
    
    # 4. 拼接
    final_embeds = torch.cat([prompt_embeds, raw_embeds], dim=1)
    
    # 5. 构造 Mask
    final_mask = torch.cat([
        torch.ones((batch_size, 8)).to(device),
        ipt['attention_mask']
    ], dim=1)

    print("开始生成...")

    # 6. 生成
    output = peft_model.generate(
        inputs_embeds=final_embeds,      
        attention_mask=final_mask,       
        max_new_tokens=128,
        do_sample=True,
        use_cache=True 
    )

    print(tokenizer.decode(output[0], skip_special_tokens=True))

开始生成...
考试技巧有很多啊，首先就是多做题，多做题我们就要知道怎么做题，做一做题我们就可以明白考题到底是什么，能掌握这些，我们就能了解答题的思路。我们还要把时间分配好，考前的时间一定要利用好，这样我们能在考试的时候稳稳的拿个分数。
